<img src="./images/DLI_Header.png" style="width: 400px;">

## 评估
恭喜您完成了今天的课程！希望您在此过程中学到了一些有价值的技能。现在该测试一下这些技能了。在此评估中，您将训练一种能够识别新鲜和腐烂水果的新模型。您需要使模型的验证准确率达到92％，才能通过评估，但我们鼓励您挑战更高的准确率。为此，您将使用先前练习中学到的技能，具体来说，我们建议您结合使用迁移学习、数据扩充和模型微调。训练好模型并在测试数据集上的准确率达到至少92％之后，请保存模型，然后评估其准确率。让我们开始吧！

### 数据集
在本练习中，您将训练一个模型来识别新鲜和腐烂的水果，数据集来自[Kaggle](https://www.kaggle.com/sriramr/fruits-fresh-and-rotten-for-classification)。如果您有兴趣在课后自己开始一个新的项目，那么Kaggle是一个值得访问的好地方。现在您可详细查看`fruits`文件夹中的数据集结构。水果有六类：新鲜的苹果，新鲜的橙子，新鲜的香蕉，烂的苹果，烂的橙子和烂的香蕉。这意味着您的模型将需要有6个神经元的输出层才能成功进行分类 您还需要使用`categorical_crossentropy`作为损失函数来编译模型，因为我们有两个以上的类别。

<img src="./images/fruits.png" style="width: 600px;">

### 加载ImageNet预训练的基础模型
我们鼓励您从在ImageNet上预训练的模型开始。您需要用正确的权重加载模型，设置输入的形状，然后选择删除模型的最后一层。请记住，图像具有三个维度：高度和宽度以及多个颜色通道。因为这些图片是彩色的，所以会有红色，绿色和蓝色三个通道。我们已经为您填写了输入形状，请不要更改，否则评估将失败。如果您需要预训练模型的参考设置，请查看笔记本05b，您在那里最先实现的迁移学习。

In [ ]:
from tensorflow import keras

base_model = keras.applications.VGG16(
    weights=FIXME,
    input_shape=(224, 224, 3),
    include_top=FIXME)

### 冻结基础模型
接下来，我们建议您像在笔记本05b中一样冻结基础模型。这样做是为了使从ImageNet数据集中所学到的知识都不会在初始的训练中被破坏。

In [ ]:
# Freeze base model
base_model.trainable = FIXME

### 向模型添加新层
现在该向预训练模型中添加新层了。您可以再次使用笔记本05b作为指导。请密切注意最后的全连接（Dense）层，并确保其具有正确数量的神经元以对不同类型的水果进行分类。

In [ ]:
# Create inputs with correct shape
inputs = FIXME

x = base_model(inputs, training=False)

# Add pooling layer or flatten layer
x = FIXME

# Add final dense layer
outputs = keras.layers.Dense(FIXME, activation = 'softmax')(x)

# Combine inputs and outputs to create model
model = keras.Model(FIXME)

In [ ]:
model.summary()

### 编译模型
现在可以使用损失函数（loss）和衡量标准（metrics）选项来编译模型了。请记住，我们正在训练的模型是要解决多分类而不是二分类的问题。

In [ ]:
model.compile(loss = FIXME , metrics = FIXME)

### 扩充数据
如果需要，请尝试扩充数据以改进数据集。请参考笔记本04a和笔记本05b中的数据扩充的示例。您也可以查看[Keras ImageDataGenerator类](https://keras.io/api/preprocessing/image/#imagedatagenerator-class)的文档。 此步骤是可选的，但是您可能会发现，这对训练时能达到95％的准确率很有帮助。

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(FIXME)

### 加载数据集
现在应该加载训练和测试数据集了。您必须选择正确的文件夹以及图像的正确的`target_size`（它必须与您创建的模型的输入高度和宽度相匹配）。如果您需要参考，可以查看笔记本05b。

In [ ]:
# load and iterate training dataset
train_it = datagen.flow_from_directory(FIXME, 
                                       target_size=FIXME, 
                                       color_mode='rgb', 
                                       class_mode="categorical")
# load and iterate test dataset
test_it = datagen.flow_from_directory(FIXME, 
                                      target_size=FIXME, 
                                      color_mode='rgb', 
                                      class_mode="categorical")

### 训练模型
现在开始训练模型！将训练和测试数据集传递给`fit`函数，并设置所需的训练次数（epochs）。

In [ ]:
model.fit(FIXME,
          validation_data=FIXME,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=test_it.samples/test_it.batch_size,
          epochs=FIXME)

### 解冻模型以进行微调
如果您已经达到了92％的验证准确率，则此步是可选的。如果没有，我们建议您以很小的学习率尝试对模型进行微调。您可以再次使用笔记本05b作为参考。

In [ ]:
# Unfreeze the base model
base_model.trainable = FIXME

# Compile the model with a low learning rate
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = FIXME),
              loss = FIXME , metrics = FIXME)

In [ ]:
model.fit(FIXME,
          validation_data=FIXME,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=test_it.samples/test_it.batch_size,
          epochs=FIXME)

### 评估模型

希望您现在拥有的模型具有92％或更高的验证准确率。如果没有，您可能需要返回并对模型进行更多的训练，或者对数据增强进行调整。

对验证精度满意后，您可以通过执行以下单元格来评估模型。`evaluate`函数将返回一个元组（tuple），其中第一个值是您的损失，第二个值是您的准确率。您需要获得0.92或更高的精度值。

In [ ]:
model.evaluate(test_it, steps=test_it.samples/test_it.batch_size)

### 执行评估

请执行以下2个代码单元来评估您的结果。

**注意：** `run_assessment` 假设您的模型是以 `model` 命名的，而且您的测试数据集的名字是`test_it`。无论出于什么原因您修改了上述名字，请在下面的单元中对`run_assessment`的参数做相应的修改。

In [ ]:
from run_assessment import run_assessment

In [ ]:
run_assessment(model, test_it)

### 生成证书

如果您通过了评估，请返回课程页面（见下图）并单击Assess（评估）按钮，就会产生本课程的合格证书。

<img src="./images/assess_task.png" style="width: 800px;">